<a href="https://colab.research.google.com/github/lima-breno/deep_learning_frameworks/blob/main/pytorch_GPU(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#0- Verificar se vc tem a GPU disponível

Verifique a instalação da GPU com o comando abaixo. Se a GPU estiver disponível, você verá informações sobre o dispositivo CUDA.

In [ ]:
import torch
torch.__version__

#Primeiro vejo se tenho uma versão que suporta o cuda.

'2.6.0+cu124'

In [ ]:
#depois verifico que se nho GPU disponível no ambiente
torch.cuda.is_available()
# o resultado indica que não tenho! Api nesse caso preciso habilitar a GPU.
## Vou em Editar/Configurações de notebook/ seleciono a GPU T4 (que o plano free tem acesso)

False

In [ ]:
#agora importo novamente a biblitoteca e vejo se a GPU está valendo
import torch
torch.cuda.is_available()

True

# 1 - Instalar o PyTorch e o CUDA: (CASO EU N TENHA A VERSAO DO CUDA!)
Certifique-se de ter o PyTorch instalado e configurado corretamente com suporte ao CUDA, que é a plataforma de computação paralela da NVIDIA usada para executar cálculos em GPUs NVIDIA.

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# 2 - Inicializar Tensores na GPU:
Para tirar proveito da GPU, você precisa criar tensores diretamente na GPU. Você pode fazer isso especificando o dispositivo ao criar o tensor.

In [ ]:
#Criando o dispositivo (hardware que estarei rodando) por meio de duas formas
#Forma 01: pra ter acesso a CPU que estamos lidando
torch.device('cpu')

device(type='cpu')

In [ ]:
#Forma 02: utilizando a GPU (chamando o cuda -> é a maneira certa)
torch.device('cuda')

device(type='cuda')

In [ ]:
#Forma 03: Posso fazer uma criaçao CONDICIONAL, eu valido se tenho uma GPU, se tiver eu instacio como GPU, caso n, instancio como CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#é o melhor!

Posso criar os tensores da seguinte maneira

In [ ]:
#FORMA 01:
##Criando um tensor aleatório para demonstrar se está sendo utilizada a GPU

tensor_1 = torch.Tensor([1,2,3])

In [ ]:
tensor_1.device
#Ele está na CPU, preciso mover para a GPU

device(type='cpu')

In [ ]:
#Posso chamar um método para passar para a GPU
tensor_1.cuda()

tensor([1., 2., 3.], device='cuda:0')

In [ ]:
#Aqui ele cria uma cópia do dado que está dentro do tensor, joga essa info lá na GPU e essa cópia tem uma referencia ao dado no tensor da cpu (ideia de ponteiros).
##Só que para eu poder usar este cara eu preciso atribuir ele a alguem.


In [ ]:
#FORMA 02:
##
tensor_2 = torch.tensor([1,2,3], device = device, dtype= torch.float64)
tensor_2

tensor([1., 2., 3.], device='cuda:0', dtype=torch.float64)

Ou você pode mover um tensor existente para a GPU usando o método `to()`.

In [ ]:
tensor_on_gpu = tensor_1.to(device)
tensor_on_gpu

tensor([1., 2., 3.], device='cuda:0')

In [ ]:
#qualquer operaao com o tensor

tensor_1 = tensor_1*10
tensor_1

tensor([10., 20., 30.])

In [ ]:
#se eu tentar multiplicar escalarmente tensor na gpu com um tensor de 64 bits, dá erro! (tensores com precisoes riferentes float x double)

tensor_on_gpu @ tensor_2

RuntimeError: dot : expected both vectors to have same dtype, but found Float and Double

In [ ]:
#dá erro pq:
tensor_on_gpu.dtype, tensor_2.dtype

(torch.float32, torch.float64)

In [ ]:
#para resolver este problema eu preciso converter tudo pra float ou tudo para double (64 bits)

#PRIMEIRO TIPO DE CONVERSAO
tensor_on_gpu.double() @ tensor_2

tensor(14., device='cuda:0', dtype=torch.float64)

In [ ]:
#SEGUNDO TIPO DE CONVERSAO
tensor_on_gpu @ tensor_2.float()

tensor(14., device='cuda:0')

In [ ]:
# O uso da GPU quando comparado com a CPU gera um ganho de processamento absurdo!

# 3 - Mover Modelos para a GPU:
Se você estiver treinando um modelo, você também deve mover o modelo para a GPU. Faça isso chamando o método to() no modelo. Primeiramente, os tensores (dataset) devem estar na GPU,o que já fora feito anteriormente.


OBS: A classe n precisa ser utilizada na GPU, somente o modelo

In [ ]:
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super(MeuModelo, self).__init__()
        self.camada = nn.Linear(10, 5)

    def forward(self, x):
        return self.camada(x)

...

Isso move todos os parâmetros do modelo para a GPU.

# 4 - Treinar na GPU:
Quando você executa operações em tensores na GPU, o PyTorch automaticamente executa essas operações na GPU. Certifique-se de que os dados de entrada e os rótulos também estejam na GPU.

In [ ]:
dados_de_treino = dados_de_treino.to(device)
rotulos_de_treino = rotulos_de_treino.to(device)

# Treinar seu modelo
saida = modelo(dados_de_treino)
perda = criterio(saida, rotulos_de_treino)
perda.backward()

## Treinando um simples modelo na GPU

Vamos criar um exemplo simples com dados aleatórios, um modelo básico e treinamento na GPU usando PyTorch. Neste exemplo, usaremos um modelo de regressão linear simples.

### Definições iniciais

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Gerar dados aleatórios
np.random.seed(42)
torch.manual_seed(42)

# Dados
num_samples = 10000
input_size = 5
output_size = 1

### Criação dos dados de treino

In [ ]:
X_train = torch.rand((num_samples, input_size)).float()
y_train = 3 * X_train[:, 0] - 2 * X_train[:, 1] + 1.5 * X_train[:, 2] + torch.randn(num_samples) #gerando uma regressao linear baseada no dado

In [ ]:
#como saber a qtd de memoria que estou usando
!nvidia-smi

Sun Jul 20 15:00:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   72C    P0             32W /   70W |     150MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Definição do modelo

In [ ]:
class LinearRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.camada = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.camada(x)

In [ ]:
modelo = LinearRegression(input_size, output_size)#posso criar o modelo e colocar o .to(device) para evitar problemas na frente
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterio = nn.MSELoss()
otimizador = optim.SGD(modelo.parameters(), lr=0.01)

In [ ]:
%%time
#enviar o modelo para GPU
modelo = modelo.to(device)

#enviar dados para GPU
X_train = X_train.to(device)
y_train = y_train.to(device)

num_epochs = 1000

for epoch in range(num_epochs):
    # Forward pass
    saida = modelo(X_train)
    perda = criterio(saida, y_train)

    # Backward pass e otimização
    otimizador.zero_grad()
    perda.backward()
    otimizador.step()

    if (epoch + 1) % 100 == 0:
        print(f'Época [{epoch + 1}/{num_epochs}], Perda: {perda.item():.4f}')

Época [100/1000], Perda: 2.2640
Época [200/1000], Perda: 2.2630
Época [300/1000], Perda: 2.2621
Época [400/1000], Perda: 2.2613
Época [500/1000], Perda: 2.2607
Época [600/1000], Perda: 2.2601
Época [700/1000], Perda: 2.2596
Época [800/1000], Perda: 2.2592
Época [900/1000], Perda: 2.2589
Época [1000/1000], Perda: 2.2586
CPU times: user 7.46 s, sys: 482 ms, total: 7.94 s
Wall time: 8.1 s


In [ ]:
#Testando na CPU
modelo = LinearRegression(input_size, output_size)
device = torch.device("cpu")
criterio = nn.MSELoss()
otimizador = optim.SGD(modelo.parameters(), lr=0.01)

In [ ]:
#Considerando o modelo na CPU
%%time

#enviar dados para GPU
X_train = X_train.to(device)
y_train = y_train.to(device)

num_epochs = 1000

for epoch in range(num_epochs):
    # Forward pass
    saida = modelo(X_train)
    perda = criterio(saida, y_train)

    # Backward pass e otimização
    otimizador.zero_grad()
    perda.backward()
    otimizador.step()

    if (epoch + 1) % 100 == 0:
        print(f'Época [{epoch + 1}/{num_epochs}], Perda: {perda.item():.4f}')

Época [100/1000], Perda: 2.2807
Época [200/1000], Perda: 2.2751
Época [300/1000], Perda: 2.2711
Época [400/1000], Perda: 2.2680
Época [500/1000], Perda: 2.2657
Época [600/1000], Perda: 2.2639
Época [700/1000], Perda: 2.2625
Época [800/1000], Perda: 2.2615
Época [900/1000], Perda: 2.2606
Época [1000/1000], Perda: 2.2599
CPU times: user 4min 33s, sys: 6min 12s, total: 10min 46s
Wall time: 10min 50s


In [ ]:
#Obs: o código acima n envia para a GPU!!!!
## DEMOROU MUITO MAIS PARA RODAR NA CPU!!
# o modelo TPU tem muito mais RAM, preciso adaptar daí!

In [ ]:
# Avaliação do modelo GPU
modelo.eval()
X_test = torch.rand((10, input_size)).float().to(device)
previsoes = modelo(X_test)
print("\nPrevisões do Modelo:")
print(previsoes.detach().cpu().numpy())

Este exemplo cria dados de treinamento com uma relação linear específica, define um modelo de regressão linear, move o modelo e os dados para a GPU (se disponível), e realiza o treinamento usando a GPU. Note que este é um exemplo muito simples e pode não refletir um cenário real, mas é útil para entender o processo básico de treinamento na GPU com PyTorch.